In [2]:
import os
import psycopg2
import psycopg2.extras as extras
from io import StringIO
from datetime import datetime, timedelta
import requests
import json
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import descartes
import geopandas as gpd
from shapely.geometry import Point, Polygon
from sklearn.neighbors import BallTree, KDTree
from scipy.spatial import ConvexHull
from ipynb.fs.full.geomath_hulls import ConcaveHull
from ipynb.fs.full.functions import ball_tree_avg_distance, project_points, create_polygon, point_in_convex_hull, initial_blue_frame, create_red_frame, calculate_initial_compass_bearing
from shapely import geometry
import matplotlib.path as mpltPath
%matplotlib inline

##############################################################################################################
# Fetch Data & Build Frame from API
##############################################################################################################
my_headers = {'Authorization' : '1566394169B0EJX2MGAVKVUGGKEMKZBMND9A7VCR'}

treesurveys = requests.get('https://sherlock.aerobotics.com/developers/treesurveys/', headers=my_headers)
surveys = requests.get('https://sherlock.aerobotics.com/developers/surveys/', headers=my_headers)

treesurvey_response = treesurveys.json()
survey_response = surveys.json()

treesurvey_df = pd.DataFrame(treesurvey_response['results'])
survey_df = pd.DataFrame(survey_response['results'])

search_orchard_id = 216269
# Select the survey where the orchard_id = search_orchard_id
search_survey_id = survey_df.loc[survey_df['orchard_id'] == search_orchard_id]['id'][0]
search_survey_id

# Select the tree surveys where the survey_id = search_survey_id
treesurvey_df = treesurvey_df.loc[treesurvey_df['survey_id'] == search_survey_id]
# Drop the area, height, id, ndre, ndvi, radius, volume   columns
treesurvey_df.drop(['area', 'height', 'id', 'ndre', 'ndvi', 'radius', 'volume'], axis='columns', inplace=True)
# Add orchard_id column to the frame
treesurvey_df['orchard_id'] = search_orchard_id

##############################################################################################################
# Work out the Convex Hull for the orchard 
##############################################################################################################
points = np.column_stack((treesurvey_df.latitude,treesurvey_df.longitude))
hull = ConvexHull(points)

##############################################################################################################
# Work out the Bearing for the orchard 
##############################################################################################################
minlonrow = treesurvey_df.loc[treesurvey_df['longitude'] == treesurvey_df.longitude.min()]
maxlatrow = treesurvey_df.loc[treesurvey_df['latitude'] == treesurvey_df.latitude.max()]

minlong_lat = minlonrow.iloc[0]['latitude']
minlong_lon = minlonrow.iloc[0]['longitude']

maxlat_lat = maxlatrow.iloc[0]['latitude']
maxlat_lon = maxlatrow.iloc[0]['longitude']

bearing = calculate_initial_compass_bearing((minlong_lat,minlong_lon),(maxlat_lat,maxlat_lon))
# covert degrees to radians
b_rad = np.deg2rad(bearing)

##############################################################################################################
# Work out the avg distance metric for this orchard in metres
##############################################################################################################

orch_avg_dist, balltree_df = ball_tree_avg_distance(treesurvey_df, treesurvey_df, 5)

##############################################################################################################
# Build out the 4 point movement columns lat_up, lat_down, lon_up, lon_down
##############################################################################################################

treesurvey_df[["newlatup", "newlonup", "newlatdown", "newlondown"]] = treesurvey_df.apply(lambda x: project_points(x.latitude, x.longitude, orch_avg_dist, b_rad), axis=1, result_type="expand")

##############################################################################################################
# First Point Analysis & Setup for remaining 3
##############################################################################################################

blue_df = initial_blue_frame(treesurvey_df)

ptype = 'up'

red_df = create_red_frame(treesurvey_df, ptype)

new_avg_dist, balltree_df = ball_tree_avg_distance(blue_df, red_df, 1)

pc = np.column_stack((balltree_df.latitude,balltree_df.longitude))
balltree_df['point_check'] = [point_in_convex_hull(point, hull) for point in pc]
treesurvey_new_points = balltree_df[balltree_df.point_check]

check = treesurvey_new_points[treesurvey_new_points['distance_1'] > (orch_avg_dist / 2)]

# Create the missing Dataframe
missing_df = check[['tree_id', 'latitude', 'longitude']].copy()


##############################################################################################################
# Process Remaining Down Point Projections
##############################################################################################################

blue_df = pd.concat([blue_df, missing_df], axis=0,sort=True)

ptype = 'down'

red_df = create_red_frame(treesurvey_df, ptype)

new_avg_dist, balltree_df = ball_tree_avg_distance(blue_df, red_df, 1)

pc = np.column_stack((balltree_df.latitude,balltree_df.longitude))
balltree_df['point_check'] = [point_in_convex_hull(point, hull) for point in pc]
treesurvey_new_points = balltree_df[balltree_df.point_check]

check = treesurvey_new_points[treesurvey_new_points['distance_1'] > (orch_avg_dist / 2)]

########################
# add any new points to missing df
add_missing_df = check[['tree_id', 'latitude', 'longitude']].copy()
missing_df = pd.concat([missing_df, add_missing_df], axis=0,sort=True)

# Reset the index
missing_df = missing_df.reset_index(drop=True)
missing_df

,latitude,longitude,tree_id
0,-32.328802,18.826421,38868729
1,-32.328678,18.826671,38868707
2,-32.328898,18.825854,38869753
3,-32.328624,18.825667,38869622
4,-32.328969,18.825898,38869744


In [ ]:
track_response = {
    "missing_trees": [
        {
            "lat": -32.32889593074451,
            "lng": 18.825854732491432
        },
        {
            "lat": -32.32880053074451,
            "lng": 18.826421332473483
        },
        {
            "lat": -32.32867583074451,
            "lng": 18.82667213245002
        },
        {
            "lat": -32.32862193074451,
            "lng": 18.82566793243988
        }
    ]
}

In [3]:
tjson = missing_df[['latitude', 'longitude']]
tjson.columns = ['lat', 'lon']
json = tjson.to_json(orient='records') 
json

'[{"lat":-32.328802201,"lon":18.826420592},{"lat":-32.328677501,"lon":18.826671392},{"lat":-32.328897601,"lon":18.825853992},{"lat":-32.328623601,"lon":18.825667192},{"lat":-32.328968999,"lon":18.8258978079}]'